In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import math
import torch.nn.functional as F
from torch.autograd import Variable
from tqdm.autonotebook import tqdm
print(torch.__version__)
import pandas as pd
plt.ion()  
import cv2
from PIL import Image
from torch.utils.data import Dataset, DataLoader

1.4.0a0+d22f614


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from efficientnet_pytorch import EfficientNet

In [4]:
def imread(path, fast_mode=False):
    img = cv2.imread(path)
    if not fast_mode and img is not None and len(img.shape) == 3:
        img = np.array(img[:, :, ::-1])
    return img

In [5]:
class CarModelNet(nn.Module):
    def __init__(self, n_classes):
        super(CarModelNet, self).__init__()
        self.base_model = models.vgg16(pretrained=True)
        for param in self.base_model.parameters():
            param.requires_grad = False
        
        self.fc1 = nn.Linear(1000, 256)
        self.drop = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(262, n_classes)
    
    def forward(self, x1, x2):
        x = F.relu(self.base_model(x1))
        x = self.drop(self.fc1(x))
        x = torch.cat([x, x2], dim=1)
        
        return F.log_softmax(x) 
    

In [6]:
def trans_data(df_train):
    t = []
    for i in df_train.ImageId.values:
        data = df_train.loc[df_train.ImageId == i,'PredictionString'].values[0]
        data = np.array(data.split(" ")).reshape(-1, 7)
        for x in data:
            d = [i] + [float(a) for a in x]
            t.append(d)
    t = pd.DataFrame(t)
    t.columns = ['ImageId', 'label', 'x', 'y', 'z', 'yaw', 'pitch', 'roll']
    return t

In [22]:
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
class CarDataset(Dataset):
    """Car dataset."""

    def __init__(self, dataframe, root_dir, training=True, transform=None):
        self.df = dataframe
        self.root_dir = root_dir
        self.transform = transform
        self.training = training

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        # Get image name
        imageId = self.df.loc[idx].ImageId
        img_name = self.root_dir.format(imageId)
        label = self.df.loc[idx].label
 
        # Read image
        img0 = imread(img_name, True)
        img0 = Image.fromarray(img0)
        
        if self.training:
            x1 = self.transform['train'](img0)
        else:
            x1 = self.transform['validation'](img0)
        
        x2 = self.df.loc[:, ('x', 'y', 'z', 'yaw', 'pitch', 'roll')].loc[0].values
        x2 = torch.from_numpy(x2)
        return [x1, x2, label]

In [8]:
PATH = "./baselines/pku-autonomous-driving/"
train_images_dir = PATH + 'train_images/{}.jpg'

In [9]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(5),
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(224, scale=(0.96, 1.0), ratio=(0.95, 1.05)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.Resize([224,224]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [10]:
train = pd.read_csv(PATH + "train.csv")

In [11]:
df_train, df_dev = train_test_split(train, test_size=0.1, random_state=42)

In [12]:
df_train = trans_data(df_train)
df_dev = trans_data(df_dev)

In [13]:
classes = list(set(df_dev.label).union(df_train.label))
n_classes = len(set(df_dev.label).union(df_train.label))
class_map = dict()
label = [x for x in range(n_classes)]
for x in zip(label, classes):
    class_map[x[1]] = x[0]

In [14]:
df_train['label'] = df_train['label'].apply(lambda x: class_map[x])
df_dev['label'] = df_dev['label'].apply(lambda x: class_map[x])

In [23]:
train_dataset = CarDataset(df_train, train_images_dir, training=True, transform=data_transforms)
dev_dataset = CarDataset(df_dev, train_images_dir, training=False, transform=data_transforms)
# test_dataset = CarDataset(df_test, test_images_dir, training=False, transform=data_transforms)

In [24]:
BATCH_SIZE = 2
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
dev_loader = DataLoader(dataset=dev_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [26]:
model = CarModelNet(n_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [27]:
def train(epochs, network, train_loader, val_loader, optimizer, use_gpu):
    """
    Fit the model to the training dataset and save the model with 
    the best validation accuracy
    
    @param epochs: number of  iteration to run through whole dataset 
    @param network: model to fit
    @param train_loader: training data
    @param val_loader: validation data
    @param optimizer: ex. Adam, SGD
    
    """
    
    

    for epoch in range(1, epochs +1):

        progress = tqdm(enumerate(train_loader), desc="Loss: ", total=len(train_loader))
        for idx, sample in enumerate(train_loader):
            x1, x2, target = sample[0], sample[1], sample[2]
            network.train()
            x1, x2 = Variable(x1), Variable(x2.type(dtype))
            target = Variable(target)
            if torch.cuda.is_available() and use_gpu:
                x1, x2 = x1.cuda(), x2.cuda()
                target = target.cuda()
                network.cuda()

            # use the cross-entropy loss
            criterion = nn.CrossEntropyLoss()
            optimizer.zero_grad()
            output = network(x1, x2)

            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            progress.set_description("Loss: {:.4f}".format(loss.item()/(4)))
            progress.update(1)
        
        # check the validation accuracy 
        loss, acc = test(network, val_loader, use_gpu)
        print("epoch: {} validation loss: {} validation accuracy: {}".format(epoch, loss, acc))


In [28]:
def test(network, loader, use_gpu):
    """
    Give the loader determine the accuracy
    
    @param loader: val_loader (validation dateset) or test_loader (test dataset)
    
    """
    
    # this stop the dropout from happening
    network.eval()
    loss = 0
    data_size = 0
    score = []
    with torch.no_grad():
        for x1, x2, target in loader:
            x1, x2 = Variable(x1), Variable(x2.type(dtype))
            target = Variable(target)
            if torch.cuda.is_available() and use_gpu:
                x1, x2 = x1.cuda(), x2.cuda()
                target = target.cuda()
                network.cuda()

            output = network(x1, x2)
            
            criterion = nn.CrossEntropyLoss()
            loss += criterion(output, target)
            
            _, pred = torch.max(output, 1)
            data_size += len(pred)
            # keep the number of the correct number of correct 
            # prediction
            score.append(pred.eq(target.data.view_as(pred)).sum().item() / len(pred))

    return loss.item()/data_size, np.array(score).mean()

In [21]:
train(1, model, train_loader, dev_loader, optimizer, True)

AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/adam/anaconda3/envs/pku/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/adam/anaconda3/envs/pku/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/adam/anaconda3/envs/pku/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-7-e4384355b81c>", line 33, in __getitem__
    self.df[col] = self.df[col].type(dtype)
  File "/home/adam/anaconda3/envs/pku/lib/python3.7/site-packages/pandas/core/generic.py", line 5179, in __getattr__
    return object.__getattribute__(self, name)
AttributeError: 'Series' object has no attribute 'type'


In [29]:
a = next(iter(train_loader))

In [ ]:
class CarModelNet(nn.Module):
    def __init__(self, n_classes):
        super(CarModelNet, self).__init__()
        self.base_model = models.vgg16(pretrained=True)
        for param in self.base_model.parameters():
            param.requires_grad = False
        
        self.fc1 = nn.Linear(1000, 256)
        self.drop = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(262, n_classes)
    
    def forward(self, x1, x2):
        x = F.relu(self.base_model(x1))
        x = self.drop(self.fc1(x))
        x = torch.cat([x, x2], dim=1)
        
        return F.log_softmax(x) 

In [30]:
k = models.vgg16(pretrained=True)

In [42]:
x1 = Variable(a[0]).cuda()

In [43]:
x = k(x1)

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [32]:
x = F.relu(x)

In [33]:
x = nn.Dropout(p=0.5)(nn.Linear(1000, 256)(x))

In [32]:
x.shape

torch.Size([2, 256])

In [34]:
torch.cat([x, a[1].type(dtype)], dim=1)

RuntimeError: Expected object of backend CUDA but got backend CPU for sequence element 0 in sequence argument at position #1 'tensors'

In [ ]:
a = imread(train_images_dir.format(df_dev.loc[0].ImageId), True)

In [ ]:
df_dev.loc[:, ('x', 'y', 'z', 'yaw', 'pitch', 'roll')].loc[0].values

In [35]:
a[1].type(dtype)

tensor([[ 0.1503, -3.0918, -3.1250, -6.4665,  3.4920, 13.0053],
        [ 0.1503, -3.0918, -3.1250, -6.4665,  3.4920, 13.0053]],
       device='cuda:0')

In [36]:
x

tensor([[-0.0000e+00, -7.1266e-01,  1.9333e+00,  2.0834e+00,  8.4949e-01,
         -0.0000e+00, -7.2108e+00, -0.0000e+00, -8.8557e-01, -7.2765e-01,
          0.0000e+00, -0.0000e+00, -0.0000e+00,  0.0000e+00, -7.8229e-01,
         -2.9613e+00, -2.3337e+00,  0.0000e+00,  0.0000e+00,  2.1171e+00,
         -0.0000e+00,  0.0000e+00, -0.0000e+00,  3.5757e-01,  8.7202e-01,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -0.0000e+00,
         -3.7323e+00, -4.6111e-01,  0.0000e+00,  0.0000e+00, -0.0000e+00,
          0.0000e+00, -1.0020e+00, -0.0000e+00, -1.4251e+00,  2.5165e+00,
         -1.0077e+00,  7.2865e-01,  1.1422e+00, -0.0000e+00,  5.7599e-01,
         -0.0000e+00, -2.3916e+00, -1.5905e+00, -7.3147e-01,  0.0000e+00,
         -2.8286e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -7.4345e-01,
         -0.0000e+00,  1.2671e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
         -0.0000e+00, -1.3494e+00, -0.0000e+00,  2.0026e+00, -8.6210e-01,
          7.3299e+00,  1.2931e+00,  3.